# Week 8

Author: Khushee Kapoor

Registration Number: 200968052

To get start, we import the required libraries and create the Frozen Lake Environment.

In [ ]:
# importing the libraries and creating the environment
import gym
import numpy as np
env = gym.make("FrozenLake-v1")

/usr/local/lib/python3.9/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.9/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


Then, we define the function to implement policy iteration. This takes in the following parameters:

- Policy
- Environment
- Discount Factor
- Theta
- Max Iterations

In [ ]:
# policy iteration
def policy_iteration(policy, env, discount_factor, theta, max_iterations):

    num_s = env.observation_space.n
    num_act = env.action_space.n
    val_func = np.zeros(num_s)
    
    for i in range(max_iterations):
        while True:
            d = 0
            
            for s in range(num_s):
                old_val = val_func[s]
                act_p = policy[s]
                act_val = np.zeros(num_act)
                
                for a in range(num_act):
                    for p, next, reward, done in env.P[s][a]:
                        act_val[a] += p*(reward + discount_factor * val_func[next])
                
                val_func[s] = np.sum(act_p * act_val)
                d = max(d, np.abs(old_val - val_func[s]))
            
            if d < theta:
                break
        
        policy_stable = True

        for s in range(num_s):
            old_act = np.argmax(policy[s])
            act_val = np.zeros(num_act)
            
            for a in range(num_act):
                for p, next, reward, done in env.P[s][a]:
                    act_val[a] += p*(reward + discount_factor * val_func[next])
            
            best_act = np.argmax(act_val)
            
            if old_act != best_act:
                policy_stable = False
            
            policy[s] = np.eye(num_act)[best_act]
        
        if policy_stable:
            break
    
    return policy

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Then, we define the function to implement value iteration. This takes in the following parameters:

- Environment
- Discount Factor
- Theta
- Max Iterations

In [ ]:
# value iteration
def value_iteration(env, discount_factor, theta, max_iterations):

    num_s = env.observation_space.n
    num_act = env.action_space.n
    val_func = np.zeros(num_s)
    
    for i in range(max_iterations):
        d = 0

        for s in range(num_s):
            old_val = val_func[s]
            act_val = np.zeros(num_act)

            for a in range(num_act):
                for p, next, reward, done in env.P[s][a]:
                    act_val[a] += p*(reward + discount_factor * val_func[next])

            val_func[s] = np.max(act_val)
            d = max(d, np.abs(old_val - val_func[s]))

        if d < theta:
            break
    
    policy = np.zeros((num_s, num_act))
    for s in range(num_s):
        act_val = np.zeros(num_act)

        for a in range(num_act):
            for p, next, reward, done in env.P[s][a]:
                act_val[a] += p*(reward + discount_factor * val_func[next])

        best_act = np.argmax(act_val)
        policy[s][best_act] = 1.0
    
    return policy

Next, we define a functions to implement the policy iteration and value iteration functions for the given number of episodes.

In [ ]:
# implementing the two techniques
def episodes(policy, env, episodes):
    tot_reward = 0
    wins = 0

    for i in range(episodes):
        state = env.reset()
        ep_reward = 0
        flag = False
        
        while not flag:
            act = np.random.choice(env.action_space.n, p=policy[state])
            next, reward, flag, _ = env.step(act)
            ep_reward += reward
            state = next
        tot_reward += ep_reward
        
        if ep_reward == 1:
            wins += 1
    
    avg_reward = tot_reward/episodes

    return wins, avg_reward

Then, we define the parameters which are passed to the functions.

In [ ]:
# defining the parameters
n_states = env.observation_space.n
n_actions = env.action_space.n
discount_factor = 0.5
theta = 1e-8
max_iterations = 1000
n_episode = 1000

Using the policy iteration function, we first pass an initial policy and then obtain the optimal policy using the policy iteration function.

In [ ]:
# optimal policy from policy iteration
policy = np.ones([n_states, n_actions]) / n_actions
opt_policy_pi = policy_iteration(policy, env, discount_factor, theta, max_iterations)
wins_pi, avg_reward_pi = episodes(opt_policy_pi, env, n_episode)

Using the value iteration function, we first pass an initial policy and then obtain the optimal policy using the value iteration function.

In [ ]:
# optimal policy from value iteration
opt_policy_vi = value_iteration(env, discount_factor, theta, max_iterations)
wins_vi, avg_reward_vi = episodes(opt_policy_vi, env, n_episode)

Printing the parameters.

In [ ]:
# printing the parameters
print("Policy Iteration:", f"Number of wins = {wins_pi}", f"Average Return = {avg_reward_pi}","", sep = '\n')
print("Value Iteration:", f"Number of wins = {wins_vi}", f"Average Return = {avg_reward_vi}","", sep = '\n')

Policy Iteration:
Number of wins = 458
Average Return = 0.458

Value Iteration:
Number of wins = 455
Average Return = 0.455



We observe that for policy iteration, it has a higher number of wins and hence higher average reward. This implies that policy iteration may have started to converge to the optimal policy faster than value iteration. This is applicable only when the discount factor is 0.5 and the results may change for different values of discount factor. However, the difference is not significant.

Thus, from the results from this notebook, (at a discount factor of 0.5, max iterations of 1000) policy iteration is the better choice to solve the FrozenLake-v1 enviorment. 